In [1]:
import random
import os
from tqdm.notebook import tqdm
import gc
from torch.nn import Parameter
import torch.nn.functional as F
import torch.nn as nn
import math
import timm
import pandas as pd
import torch
import numpy as np
from torch.amp import GradScaler
import cv2
import random
from tqdm import tqdm
from torch.autograd import Variable
from skimage.metrics import structural_similarity as ssim

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(228)

In [3]:
path2files = '/kaggle/input/ioai-contest-2/'

pairs = pd.read_csv(path2files + 'imgs/pairs_list.csv')
paths_embeds = pd.read_csv(path2files + 'imgs/paths_embeds.csv')['image_path']
real_embeds = np.load(path2files + 'imgs/real_embeds.npy')

In [4]:
class MCSDataset(torch.utils.data.Dataset):
    def __init__(self, image_path, target, imsize = 112):
        self.image_path = image_path
        self.target = target
        self.image_size = imsize

    def __len__(self):
        return len(self.target)

    def resize(self, img, interp):
        return  cv2.resize(
            img, (self.image_size, self.image_size), interpolation=interp)

    def __getitem__(self, idx):
        path = self.image_path[idx]
        target = self.target[idx]
        img = cv2.imread(f'{path2files}imgs/train/{path}')

        img = self.resize(img, cv2.INTER_LINEAR)

        img = (img / 255.) - 0.5
        img = np.transpose(img,(2,0,1)).astype(np.float32)
        img = torch.from_numpy(img)
        target = torch.from_numpy(target)

        return img, target

In [5]:
def make_predict(model, val_loader, loss_func, DEVICE = 'cuda'):
    preds = []
    targets = []
    model.eval()
    average_loss = 0
    with torch.no_grad():
        for batch_number,  (img, target) in enumerate(val_loader):
            img = img.to(DEVICE)
            target = target.to(DEVICE)

            with torch.amp.autocast('cuda'):
                outputs = model(img)
                loss = loss_func(outputs, target)

            average_loss += loss.cpu().detach().numpy()
            preds += [outputs.to('cpu').numpy()]
            targets += [target.to('cpu').numpy()]
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)

    print('MSE: ', ((preds -  targets) ** 2).mean())

In [6]:
class Model(nn.Module):
    def __init__(self, model_name, num_unfrozen_layers=0):
        super().__init__()
        self.model_name = model_name
        self.timm_ = timm.create_model(model_name, global_pool='', num_classes=0, in_chans=3)
        
        # for param in self.timm_.parameters():
        #     param.requires_grad = False
        
        # total_layers = len(list(self.timm_.children()))
        # for i, layer in enumerate(self.timm_.children()):
        #     if i >= total_layers - num_unfrozen_layers:
        #         for param in layer.parameters():
        #             param.requires_grad = True
        
        output_features = self.timm_(torch.zeros((1, 3, 112, 112))).shape[1]
        
        self.norm = nn.BatchNorm1d(output_features)
    
    def forward(self, x):
        out_ = self.timm_(x).mean(dim=(2, 3))
        
        out_ = self.norm(out_)
        out_ = F.normalize(out_)
        return out_

In [7]:
gc.collect()
torch.cuda.empty_cache()

In [8]:
batch_size = 8
valid_batch_size = 64
epochs = 10
lr = 0.001
num_models = 10
DEVICE = 'cuda'

params_train = {'batch_size': batch_size, 'shuffle': True, 'drop_last': True, 'num_workers': 4}
params_val = {'batch_size': batch_size, 'shuffle': False, 'drop_last': False, 'num_workers': 4}

In [9]:
def split_global_data(paths, targets, val_ratio=0.2):
    indices = list(range(len(paths)))
    random.shuffle(indices)
    split_idx = int(len(indices) * (1 - val_ratio))
    train_indices, final_val_indices = indices[:split_idx], indices[split_idx:]
    
    train_paths = [paths[i] for i in train_indices]
    train_targets = [targets[i] for i in train_indices]
    final_val_paths = [paths[i] for i in final_val_indices]
    final_val_targets = [targets[i] for i in final_val_indices]
    
    return train_paths, train_targets, final_val_paths, final_val_targets

In [10]:
train_path, train_target, final_val_path, final_val_target = split_global_data(paths_embeds, real_embeds, val_ratio=0.05)

In [11]:
final_val_loader = torch.utils.data.DataLoader(MCSDataset(final_val_path, final_val_target), **params_val)

In [12]:
def split_data_for_models(paths, targets, num_models, val_ratio=0.2):
    model_splits = []
    indices = list(range(len(paths)))
    random.shuffle(indices)
    
    for _ in range(num_models):
        split_idx = int(len(indices) * (1 - val_ratio))
        train_indices, val_indices = indices[:split_idx], indices[split_idx:]
        
        train_paths = [paths[i] for i in train_indices]
        train_targets = [targets[i] for i in train_indices]
        val_paths = [paths[i] for i in val_indices]
        val_targets = [targets[i] for i in val_indices]
        
        model_splits.append((train_paths, train_targets, val_paths, val_targets))
        random.shuffle(indices)
    
    return model_splits

In [13]:
model_splits = split_data_for_models(train_path, train_target, num_models=num_models)

In [14]:
models = []
train_loaders = []
val_loaders = []

for model_idx, (train_paths, train_targets, val_paths, val_targets) in enumerate(model_splits):
    train_loader = torch.utils.data.DataLoader(MCSDataset(train_paths, train_targets), **params_train)
    train_loaders.append(train_loader)
    
    val_loader = torch.utils.data.DataLoader(MCSDataset(val_paths, val_targets), **params_val)
    val_loaders.append(val_loader)
    
    model = Model('resnet18').to(DEVICE)
    models.append(model)

In [15]:
loss_func = torch.nn.MSELoss()

In [16]:
for model_idx, model in enumerate(models):
    print(f"Training model {model_idx + 1}/{num_models}")
    optimizer = torch.optim.AdamW(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loaders[model_idx]) * epochs, 0.00001)
    scaler = GradScaler()

    for epoch in range(epochs):
        model.train()
        average_loss = 0
        for batch_number, (img, target) in enumerate(train_loaders[model_idx]):
            optimizer.zero_grad()
            img = img.to(DEVICE)
            target = target.to(DEVICE)

            with torch.amp.autocast('cuda'):
                outputs = model(img)
                loss = loss_func(outputs, target)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            average_loss += loss.cpu().detach().numpy()

        make_predict(model, val_loaders[model_idx], loss_func)

        print(f"Epoch {epoch + 1}, Model {model_idx + 1}, Average Loss: {average_loss / len(train_loaders[model_idx])}")

Training model 1/10
MSE:  0.0024043827
Epoch 1, Model 1, Average Loss: 0.0030331774004489966
MSE:  0.0020077734
Epoch 2, Model 1, Average Loss: 0.0022495758580043912
MSE:  0.001811279
Epoch 3, Model 1, Average Loss: 0.0019489930970839372
MSE:  0.0016930894
Epoch 4, Model 1, Average Loss: 0.0017399918074768625
MSE:  0.0016203463
Epoch 5, Model 1, Average Loss: 0.0015675658853969683
MSE:  0.0015687458
Epoch 6, Model 1, Average Loss: 0.0014208102406394717
MSE:  0.001544844
Epoch 7, Model 1, Average Loss: 0.0012943903246502343
MSE:  0.001534606
Epoch 8, Model 1, Average Loss: 0.0011919230236181696
MSE:  0.0015299467
Epoch 9, Model 1, Average Loss: 0.0011274391931702236
MSE:  0.0015318071
Epoch 10, Model 1, Average Loss: 0.001094206698655494
Training model 2/10
MSE:  0.0024316614
Epoch 1, Model 2, Average Loss: 0.0030523640561946914
MSE:  0.0020139378
Epoch 2, Model 2, Average Loss: 0.0022641887697146127
MSE:  0.0018223898
Epoch 3, Model 2, Average Loss: 0.001962200278465293
MSE:  0.0017096

In [17]:
def ensemble_predict_final(models, loader, targets, loss_func):
    all_models_outputs = []
    with torch.no_grad():
        for model in models:
            model.eval()
            model_outputs = []
            for img, _ in loader:
                img = img.to(DEVICE)
                outputs = model(img)
                model_outputs.append(outputs.cpu())
            all_models_outputs.append(torch.cat(model_outputs))

    ensemble_outputs = torch.mean(torch.stack(all_models_outputs), dim=0)
    targets = torch.tensor(targets)

    loss = loss_func(ensemble_outputs, targets)
    print(f"Final Ensemble Validation Loss: {loss.item()}")

In [18]:
ensemble_predict_final(models, final_val_loader, final_val_target, loss_func)

Final Ensemble Validation Loss: 0.0011864586267620325


<ipython-input-17-f0789949591c>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  targets = torch.tensor(targets)


## Attack

In [19]:
def read_img(path, image_size=112):
    img = cv2.imread(f'{path2files}imgs/train/{path}')
    img_ = cv2.resize(
        img, (image_size, image_size), interpolation=cv2.INTER_LINEAR)
    img = (img_ / 255.) - 0.5
    img = np.transpose(img, (2, 0, 1)).astype(np.float32)
    img = torch.from_numpy(img)
    return img, img_

In [20]:
def ensemble_predict(models, img):
    with torch.no_grad():
        outputs = [model(img).cpu() for model in models]
        ensemble_output = torch.mean(torch.stack(outputs), dim=0)
    return ensemble_output

In [21]:
max_iter = 10
loss = nn.MSELoss()
eps = 1e-3
attacked_img_dict = {}

for sour, targ in tqdm(zip(pairs['source_imgs'], pairs['target_imgs']), total=len(pairs)):
    target_descriptors = np.ones((5, 512), dtype=np.float32)
    targ = targ.split('|')
    sour = sour.split('|')

    list_tagt_img = []
    for i, t in enumerate(targ):
        img, orig_tgt = read_img(t)
        list_tagt_img += [orig_tgt]
        img = img.unsqueeze(0).cuda(non_blocking=True)
        
        res = ensemble_predict(models, img).numpy().squeeze()
        target_descriptors[i] = res

    for ii, s in enumerate(sour): 
        img, orig_img = read_img(s)
        img = img.unsqueeze(0).cuda(non_blocking=True)
        input_var = torch.autograd.Variable(img, requires_grad=True)
        attacked_img = orig_img
        
        for iter_number in range(max_iter):
            adv_noise = torch.zeros((3, 112, 112)).cuda(non_blocking=True)
            
            for tg in target_descriptors:
                target_out = torch.autograd.Variable(torch.from_numpy(tg).unsqueeze(0).cuda(non_blocking=True), requires_grad=False)
                
                calc_loss = 0
                for model in models:
                    input_var.grad = None
                    out = model(input_var)
                    calc_loss += loss(out, target_out)
                
                calc_loss.backward()
                
                noise = eps * torch.clamp(input_var.grad.data / input_var.grad.data.std(), -2, 2)
                adv_noise = adv_noise + noise
            
            input_var.data = input_var.data - adv_noise
            
            changed_img = input_var.data.cpu().squeeze()
            changed_img = ((changed_img + 0.5) * 255)
            changed_img[changed_img < 0] = 0
            changed_img[changed_img > 255] = 255
            changed_img = np.transpose(changed_img.numpy(), (1, 2, 0)).astype(np.int16)
            
            ssim_score = ssim(orig_img, changed_img, channel_axis=2, data_range=256)
            if ssim_score < 0.95:
                break
            else:
                attacked_img = changed_img
        
        attacked_img_dict[s] = attacked_img

100%|██████████| 1000/1000 [5:11:41<00:00, 18.70s/it]


In [22]:
sample_submission = pd.read_csv(path2files + 'imgs/sample_submission.csv')
sample_submission_df = pd.DataFrame()
sample_submission_df['Id'] = sample_submission['Id']

result = []
for id_ in tqdm(sample_submission_df['Id']):
    result += ['|'.join([str(i) for i in attacked_img_dict[id_].flatten().tolist()])]
sample_submission_df['Target'] = result

100%|██████████| 5000/5000 [00:31<00:00, 158.58it/s]


In [23]:
sample_submission_df.to_csv('submission_ensemble.csv', index=None)

In [24]:
# !kaggle competitions submit -c ioai-contest-2 -f submission_ensemble.csv -m "10 resnet34 // 12.4 // 1.2878+"